<a href="https://colab.research.google.com/github/HSV-AI/presentations/blob/master/2021/210217_Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

# Quick Start for Recommendation Systems

Agenda:
- Welcome
- Project updates
- Current news
- Presentation on Recommendation Systems
- Q&A
- Close

We will start with a common dataset used for exploring recommendation systems, the [MovieLens Dataset](http://grouplens.org/datasets/movielens/)

We will also use the [Surprise](http://surpriselib.com/) library to build a few different recommendation systems and look at their accuracy for the dataset.

The name SurPRISE (roughly :) ) stands for Simple Python RecommendatIon System Engine.

In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 6.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618236 sha256=9bbcffb8d69841f02d58ecac464a09dcb37ff5d69540e311319a04fb3920aaa0
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [59]:
# Global imports
import pandas as pd
import io
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import KNNBasic
from surprise import get_dataset_dir
from surprise import dump

In [28]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')


## Looking at the data

u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.

- Each user has rated at least 20 movies.
- Users and items are numbered consecutively from 1.
- The data is randomly ordered.
- This is a tab separated list of user id | item id | rating | timestamp. 
- The time stamps are unix seconds since 1/1/1970 UTC


In [29]:
data_df = pd.read_table('/root/.surprise_data/ml-100k/ml-100k/u.data', header=None)
data_df.head()

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


u.item     -- Information about the items (movies); 
- this is a tab separated list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
- The last 19 fields are the genres, a 1 indicates the movie is of that genre, a 0 indicates it is not
- movies can be in several genres at once.
- The movie ids are the ones used in the u.data data set.

In [30]:
item_df = pd.read_csv('/root/.surprise_data/ml-100k/ml-100k/u.item', sep="|", encoding='iso-8859-1', header=None)
item_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


u.user     -- Demographic information about the users
- this is a tab separated list of
              user id | age | gender | occupation | zip code
- The user ids are the ones used in the u.data data set.

In [31]:
user_df = pd.read_csv('/root/.surprise_data/ml-100k/ml-100k/u.user', sep="|", encoding='iso-8859-1', header=None)
user_df.head()

,0,1,2,3,4
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


## Types of Recommendation Systems (From Wikipedia)

Other references used:

[Memory Based Recommendation System](https://towardsdatascience.com/how-to-build-a-memory-based-recommendation-system-using-python-surprise-55f3257b2cf4)

[Model Based Recommendation System](https://towardsdatascience.com/how-to-build-a-model-based-recommendation-system-using-python-surprise-2df3b77ab3e5)

### Collaborative Filtering

> Collaborative filtering is based on the assumption that people who agreed in the past will agree in the future, and that they will like similar kinds of items as they liked in the past. The system generates recommendations using only information about rating profiles for different users or items. By locating peer users/items with a rating history similar to the current user or item, they generate recommendations using this neighborhood. Collaborative filtering methods are classified as memory-based and model-based.

> #### Memory Based

> They are called memory-based because the algorithm is not complicated, but requires a lot of memory to keep track of the results.

> #### Model Based

> Model Based approaches build some type of machine learning model. For the surprise package, there are three models avaialble: SVD, SVDpp, and NMF.

> #### Problems with Collaborative Filtering

> - Cold start: For a new user or item, there isn't enough data to make accurate recommendations.
- Scalability: In many of the environments in which these systems make recommendations, there are millions of users and products. Thus, a large amount of computation power is often necessary to calculate recommendations.
- Sparsity: The number of items sold on major e-commerce sites is extremely large. The most active users will only have rated a small subset of the overall database. Thus, even the most popular items have very few ratings.

### Content Based Filtering

> Content-based filtering methods are based on a description of the item and a profile of the user's preferences. These methods are best suited to situations where there is known data on an item (name, location, description, etc.), but not on the user. Content-based recommenders treat recommendation as a user-specific classification problem and learn a classifier for the user's likes and dislikes based on an item's features.

## Assessing Model Performance

### Accuracy measures

- rmse	Compute RMSE (Root Mean Squared Error).
- mse	Compute MSE (Mean Squared Error).
- mae	Compute MAE (Mean Absolute Error).
- fcp	Compute FCP (Fraction of Concordant Pairs).

### Precision and Recall


![precision](https://wikimedia.org/api/rest_v1/media/math/render/svg/5b7d5cd5010efe2ef51e7731f2124a2156830fbe)

![recall](https://wikimedia.org/api/rest_v1/media/math/render/svg/43a4548e95fde15433d8e3cd3c80ced433f54abe)

An item is considered relevant if its true rating rui is greater than a given threshold. An item is considered recommended if its estimated rating r^ui is greater than the threshold, and if it is among the k highest estimated ratings.

Note that in the edge cases where division by zero occurs, Precision@k and Recall@k values are undefined. As a convention, we set their values to 0 in such cases.

One can also interpret precision and recall not as ratios but as estimations of probabilities:

- Precision is the estimated probability that a document randomly selected from the pool of retrieved documents is relevant.

- Recall is the estimated probability that a document randomly selected from the pool of relevant documents is retrieved.

Another interpretation is that precision is the average probability of relevant retrieval and recall is the average probability of complete retrieval averaged over multiple retrieval queries.

### F-Score

The F-score is a measure of a test's accuracy. It is calculated from the precision and recall of the test, where the precision is the number of correctly identified positive results divided by the number of all positive results, including those not identified correctly, and the recall is the number of correctly identified positive results divided by the number of all samples that should have been identified as positive.

![f-score](https://wikimedia.org/api/rest_v1/media/math/render/svg/9c94f59b68f5ae0dc92185906c7ec4214fd04e1e)

The highest possible value of an F-score is 1.0, indicating perfect precision and recall, and the lowest possible value is 0, if either the precision or the recall is zero

In [57]:
def get_f_score(precision, recall):
  denominator = precision + recall
  if(denominator == 0):
    return 0
  return 2 * (precision * recall) / denominator

## Memory Based

In [35]:
my_k = 15
my_min_k = 5
my_sim_option = {
    'name':'cosine', 'user_based':False, 
    }

algo = KNNBasic(
    k = my_k, min_k = my_min_k, sim_option = my_sim_option
    )

In [37]:

# Run 5-fold cross-validation and print results
results = cross_validate(algo, data, measures=['RMSE', 'MSE', 'MAE', 'FCP'], cv=5, verbose=True)

# You can get several of the items specifically from the returned results. Very
# useful if you need to wrap this in a grid search

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE, FCP of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9939  0.9753  0.9840  0.9775  0.9768  0.9815  0.0069  
MSE (testset)     0.9878  0.9512  0.9682  0.9554  0.9541  0.9633  0.0135  
MAE (testset)     0.7831  0.7667  0.7767  0.7713  0.7720  0.7740  0.0056  
FCP (testset)     0.6867  0.6964  0.7017  0.7015  0.7029  0.6978  0.0060  
Fit time          0.36    0.34    0.35    0.32    0.33    0.34    0.01    
Test time         2.64    2.61    2.71    2.71    2.63    2.66    0.04    


In [38]:
algo.predict(uid = '244', iid = '2')

Prediction(uid='244', iid='2', r_ui=None, est=3.126743581639865, details={'actual_k': 15, 'was_impossible': False})

In [46]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

trainset = data.build_full_trainset()

algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [41]:
print(predictions[0])

user: 196        item: 302        r_ui = 3.53   est = 4.16   {'actual_k': 15, 'was_impossible': False}


In [ ]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [47]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [58]:
kf = KFold(n_splits=5)

algo = KNNBasic(
    k = my_k, min_k = my_min_k, sim_option = my_sim_option
    )

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    fscore = get_f_score(precision, recall)
    print('Precision:', precision, 'Recall:', recall, 'F1 Score', fscore)

Computing the msd similarity matrix...
Done computing similarity matrix.
Precision: 0.6758058802692181 Recall: 0.26473589960499544 F1 Score 0.3804404684613679
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision: 0.7176073837415695 Recall: 0.28470541866479804 F1 Score 0.40767055979848077
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision: 0.7010084925690031 Recall: 0.27401136533911835 F1 Score 0.3940110400936066
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision: 0.6714842366277016 Recall: 0.2652149091614174 F1 Score 0.380245101367121
Computing the msd similarity matrix...
Done computing similarity matrix.
Precision: 0.6717975937721165 Recall: 0.25857221288595744 F1 Score 0.3734175146054762


In [54]:
def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid


# First, train the algortihm to compute the similarities between items
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

The 10 nearest neighbors of Toy Story are:
Beauty and the Beast (1991)
Raiders of the Lost Ark (1981)
That Thing You Do! (1996)
Lion King, The (1994)
Craft, The (1996)
Liar Liar (1997)
Aladdin (1992)
Cool Hand Luke (1967)
Winnie the Pooh and the Blustery Day (1968)
Indiana Jones and the Last Crusade (1989)


In [62]:
# Compute predictions of the 'original' algorithm.
predictions = algo.test(trainset.build_testset())

# Dump algorithm and reload it.
file_name = '/content/knn_basic_dump'
dump.dump(file_name, algo=algo)
_, loaded_algo = dump.load(file_name)

# We now ensure that the algo is still the same by checking the predictions.
predictions_loaded_algo = loaded_algo.test(trainset.build_testset())
assert predictions == predictions_loaded_algo
print('Predictions are the same')


Predictions are the same
